<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Totals.csv')
#print(skater_EV_totals_raw)
#skater_EV_totals_raw.head()

In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_EV_totals

In [4]:
#Add columns for G+/-,xG+/-
skater_EV_totals['G+/-'] = skater_EV_totals['GF'] - skater_EV_totals['GA']
skater_EV_totals['xG+/-'] = skater_EV_totals['xGF'] - skater_EV_totals['xGA']
#skater_EV_totals

In [5]:
#Drop columns we don't need
skater_EV_totals = skater_EV_totals.drop(columns = ['GF%','xGF%','GF','GA','xGF','xGA'])
#skater_EV_totals

In [6]:
#Rename columns
skater_EV_totals = skater_EV_totals.rename(columns = {'Position':'Position_EV','GP':'GP_EV','TOI':'TOI_EV','G+/-':'G_EV','xG+/-':'xG_EV'})
#skater_EV_totals

In [7]:
#Check for duplicates
#skater_EV_totals.loc[skater_EV_totals.Player=="Andrew Cogliano"]

In [8]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Totals.csv')
#print(skater_PP_totals_raw)
#skater_PP_totals_raw.head()

In [9]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_PP_totals

In [10]:
#Add columns for G+/-,xG+/-,G/s,xG/s
skater_PP_totals['G+/-'] = skater_PP_totals['GF'] - skater_PP_totals['GA']
skater_PP_totals['xG+/-'] = skater_PP_totals['xGF'] - skater_PP_totals['xGA']
#skater_PP_totals

In [11]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G+/-'].mean()
xG_mean = skater_PP_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [12]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G+/-'] - G_mean
skater_PP_totals['xGAA'] = skater_PP_totals['xG+/-'] - xG_mean
#skater_PP_totals

In [13]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['GF%','xGF%','GF','GA','xGF','xGA','G+/-','xG+/-','xGAA'])
#skater_PP_totals

In [14]:
#Rename columns
skater_PP_totals = skater_PP_totals.rename(columns = {'Position':'Position_PP','GP':'GP_PP','TOI':'TOI_PP','GAA':'GAA_PP'})
#skater_PP_totals

In [15]:
#Check for duplicates
#skater_PP_totals.loc[skater_PP_totals.Player=='Andrew Cogliano']

In [16]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Totals.csv')
#print(skater_SH_totals_raw)
#skater_SH_totals_raw.head()

In [17]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF%','xGF%','GF','GA','xGF','xGA']]
#skater_SH_totals

In [18]:
#Add columns for G+/-,xG+/-
skater_SH_totals['G+/-'] = skater_SH_totals['GF'] - skater_SH_totals['GA']
skater_SH_totals['xG+/-'] = skater_SH_totals['xGF'] - skater_SH_totals['xGA']
#skater_SH_totals

In [19]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G+/-'].mean()
xG_mean = skater_SH_totals['xG+/-'].mean()
#G_mean
#xG_mean

In [20]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G+/-'] - G_mean
skater_SH_totals['xGAA'] = skater_SH_totals['xG+/-'] - xG_mean
#skater_SH_totals

In [21]:
#Drop columns
skater_SH_totals = skater_SH_totals.drop(columns = ['xGF%','xGF','xGA','xG+/-','xGAA','GF%','GF','GA','G+/-'])
#skater_SH_totals

In [22]:
#Rename columns
skater_SH_totals = skater_SH_totals.rename(columns = {'Position':'Position_SH','GP':'GP_SH','TOI':'TOI_SH','GAA':'GAA_SH'})
#skater_SH_totals

In [23]:
#Check for duplicates
#skater_SH_totals.loc[skater_SH_totals.Player=='Andrew Cogliano']

In [24]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
#print(skater_box_totals_raw)
#skater_box_totals_raw.head()

In [25]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
#skater_box_totals.head()

In [26]:
#Check for duplicates
#skater_box_totals.loc[skater_box_totals.Player=='Andrew Cogliano']

In [27]:
#Look at positions to get correct F/D average points
#print(skater_box_totals['Position'].unique())
#print(skater_box_totals.count())

forwards_table = skater_box_totals.loc[(skater_box_totals.Position == "L") | (skater_box_totals.Position == "C") | (skater_box_totals.Position == "C/L") | (skater_box_totals.Position == "R") | (skater_box_totals.Position == "L/R") | (skater_box_totals.Position == "C/R")]
#forwards_table.head(10)
#print(forwards_table.count())

defensemen_table = skater_box_totals.loc[(skater_box_totals.Position == "D") | (skater_box_totals.Position == "D/L")]
#defensemen_table.head(10)
#print(defensemen_table.count())

In [28]:
#Check for duplicates
#forwards_table.loc[forwards_table.Player=='Andrew Cogliano']

In [29]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defensemen_mean = defensemen_table['Points'].mean()
#print(Defensemen_mean)

In [30]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine

forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

defensemen_table['PAA'] = defensemen_table['Points'] - Defensemen_mean
#defensemen_table.head(10)


#Add dataframes back together and sort again

concat_frames = [forwards_table, defensemen_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [31]:
#Combine skater EV and PP stats
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="outer", on=['Player','Season'])
#combined_player_df.head()


In [32]:
#Drop duplicate columns
skater_EV_PP = combined_player_df.drop(columns = ['Team_y','Position_PP','GP_PP'])
#skater_EV_PP

In [33]:
#Check for duplicates
#skater_EV_PP.loc[skater_EV_PP.Player=='Andrew Cogliano']

In [34]:
#Merge EV,PP,SH tables together
full_player_totals = pd.merge(skater_EV_PP, skater_SH_totals, how="left", on=["Player", "Season"])
#full_player_totals.head(10)

In [35]:
#Clean up data
full_player_totals = full_player_totals.drop(columns = ['Team','Position_SH','GP_SH'])
#full_player_totals.head(10)

In [36]:
#full_player_totals.loc[full_player_totals.Player=='Andrew Cogliano']

In [37]:
#Merge EV,PP,SH and Points tables together
full_player_totals = pd.merge(full_player_totals, skater_box_totals, how="left", on=["Player", "Season"])
#full_player_totals.head()

In [38]:
#Drop Repeated Columns
full_player_totals = full_player_totals.drop(columns = ['Team','Position','GP','G','A1','A2'])
#full_player_totals.head(10)


In [39]:
#Rename Columns
full_player_totals = full_player_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position','GP_EV':'GP','TOI':'TOI_tot'})
#full_player_totals

In [40]:
#Replace NaN with 0
full_player_totals = full_player_totals.replace(np.NaN, 0)
#full_player_totals

In [41]:
#Check for duplicates
#full_player_totals.loc[full_player_totals.Player=='Andrew Cogliano']

In [42]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
#print(goalie_stats_raw)
#goalie_stats_raw.head()

In [43]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GA','Sv%','GSAx']]
#goalie_stats.head()

In [44]:
#Add Active / Inactive Column to both data tables
full_player_totals['A/I'] = 'Active'
full_player_totals

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA,A/I
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-22.347988,Active
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535,Active
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535,Active
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535,Active
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,1,-13.995465,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,0.00,0.000000,214.07,-11.567407,1428.48,14,-0.995465,Active
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,0.00,0.000000,145.72,-6.487407,1006.27,10,-4.995465,Active
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,0.00,0.000000,123.65,4.072593,1348.45,14,-0.995465,Active
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,0.00,0.000000,147.87,-13.037407,951.77,19,-5.347988,Active


In [45]:
#List of team names
#print(full_player_totals['Team'].unique())

In [66]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
json_api_data["teams"][0]




{'abbreviation': 'NJD',
 'active': True,
 'conference': {'id': 6, 'link': '/api/v1/conferences/6', 'name': 'Eastern'},
 'division': {'abbreviation': 'M',
  'id': 18,
  'link': '/api/v1/divisions/18',
  'name': 'Metropolitan',
  'nameShort': 'Metro'},
 'firstYearOfPlay': '1982',
 'franchise': {'franchiseId': 23,
  'link': '/api/v1/franchises/23',
  'teamName': 'Devils'},
 'franchiseId': 23,
 'id': 1,
 'link': '/api/v1/teams/1',
 'locationName': 'New Jersey',
 'name': 'New Jersey Devils',
 'officialSiteUrl': 'http://www.newjerseydevils.com/',
 'roster': {'link': '/api/v1/teams/1/roster',
  'roster': [{'jerseyNumber': '45',
    'person': {'fullName': 'Jonathan Bernier',
     'id': 8473541,
     'link': '/api/v1/people/8473541'},
    'position': {'abbreviation': 'G',
     'code': 'G',
     'name': 'Goalie',
     'type': 'Goalie'}},
   {'jerseyNumber': '44',
    'person': {'fullName': 'Miles Wood',
     'id': 8477425,
     'link': '/api/v1/people/8477425'},
    'position': {'abbreviation': 

In [47]:
#Make a Data Table for each team
Ducks_skaters = full_player_totals.loc[(full_player_totals.Team == 'ANA')]
Coyotes_skaters = full_player_totals.loc[(full_player_totals.Team == 'ARI')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
Avalanche_skaters = full_player_totals.loc[(full_player_totals.Team == 'COL')]
#Avalanche_skaters.head(10)

In [48]:
full_player_totals.head(30)

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA,A/I
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-22.347988,Active
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535,Active
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535,Active
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535,Active
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,1,-13.995465,Active
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,0.00,0.000000,665.12,13,-1.995465,Active
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,0.00,0.000000,594.65,16,1.004535,Active
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,0.00,0.000000,886.05,22,7.004535,Active
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,0.00,0.000000,0.00,0.000000,54.97,3,-21.347988,Active
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,0.00,0.000000,0.00,0.000000,117.45,5,-19.347988,Active


In [49]:
#Inspect Data
full_player_totals.head(20)

,Player,Season,Team,Position,GP,TOI_EV,G_EV,xG_EV,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,Points,PAA,A/I
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,2,-22.347988,Active
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,41,26.004535,Active
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,22,7.004535,Active
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,57,42.004535,Active
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,1,-13.995465,Active
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,0.00,0.000000,665.12,13,-1.995465,Active
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,0.00,0.000000,594.65,16,1.004535,Active
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,0.00,0.000000,886.05,22,7.004535,Active
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,0.00,0.000000,0.00,0.000000,54.97,3,-21.347988,Active
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,0.00,0.000000,0.00,0.000000,117.45,5,-19.347988,Active


**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [50]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_player_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_EV', 'G_EV', 'xG_EV',
       'TOI_PP', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI_tot', 'Points', 'PAA',
       'A/I'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.